In [14]:
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Aref\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression ,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib
import streamlit as st
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv(r'D:\MachineLerningMaktabkhoone-karbordi\Projects\8-divar\last\tehranhouses.csv')

In [15]:
#def lower_upper(x) :
#    Q1 = np.percentile(x,25)
#    Q3 = np.percentile(x,75)
#    IQR = Q3-Q1
#    lower = Q1 - 1.5*IQR
#    upper = Q3 + 1.5*IQR
#    return lower,upper
#lower_area , upper_area = lower_upper( df['Area'] )
#lower_price , upper_price = lower_upper( df['Price'] )

In [16]:
#area_outliers = np.where(df[ (df.Area < lower_area) |(df.Area > upper_area) ])
#price_outliers = np.where(df[ (df.Area < upper_price) |(df.Area > upper_price) ])

In [17]:
#df.drop( df['Area'] == area_outliers )
#df.drop( df['Price']== price_outliers )

In [6]:
df.dropna(inplace=True)
df_final = pd.get_dummies(df['Address'],dtype=int)
df_final = df.merge(df_final,left_index=True,right_index=True)
df_final.drop(columns = 'Address', inplace = True)

In [7]:
boolean_features = ['Parking','Warehouse','Elevator']
df_final[boolean_features] =df_final[boolean_features].astype('int64')

In [8]:
df_final['Area'] = df_final['Area'].apply(lambda x : re.sub('[^\d]','',x))
df_final['Area'] = pd.to_numeric(df_final['Area'],errors='coerce')
if '3,600' in df_final['Price'] :
    df_final = df_final.drop(df_final[df_final['Price']=='3,600'].index)

In [9]:
y = df_final['Price']
df_final.drop(columns=['Price','Price(USD)'],inplace=True)
x = df_final

In [10]:
columns =x.columns
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [11]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [12]:
def parameters_finder(model,parameters) :

    grid = GridSearchCV(model , param_grid=parameters , refit=True ,cv = KFold(shuffle=True,random_state=1) , n_jobs= -1)
    grid_fit = grid.fit(x_train,y_train)    
    return grid_fit